# Music Genre Classification with GTZAN Dataset

## Overview
This notebook implements a music genre classification system using the GTZAN dataset. The goal is to classify audio clips into 10 genres (e.g., Blues, Classical, Rock) using two approaches: a **tabular approach** with pre-extracted audio features and an **image-based approach** with spectrograms processed by a convolutional neural network (CNN). The task compares these approaches to evaluate their performance and explores transfer learning as a bonus.

## Dataset
The GTZAN dataset contains:
- **Audio Files**: 1000 WAV files (30 seconds each) across 10 genres in the `genres_original` directory.
- **Features**: Pre-extracted features (e.g., MFCCs, chroma, tempo) in `features_30_sec.csv` for tabular classification.

## Approaches
1. **Tabular Approach**: Uses pre-extracted features with a dense neural network to classify genres.
2. **Image-Based Approach**: Generates Mel-spectrograms from audio files and trains a CNN for classification.
3. **Comparison**: Evaluates both approaches using accuracy, classification reports, and interactive confusion matrices.

## Tools
- **Python**: Core programming language.
- **Librosa**: For audio processing (MFCCs, spectrograms).
- **Scikit-learn**: For preprocessing and metrics.
- **Keras/TensorFlow**: For neural network models.
- **Plotly**: For interactive visualizations.

## Goals
- Achieve high classification accuracy for both approaches.
- Compare tabular and image-based models to understand their strengths and weaknesses.
- Optionally explore transfer learning (e.g., VGG16) for the image-based approach.

---

## Imports
Import all necessary Python libraries here.

---


import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras import layers, models
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
import sklearn.metrics as skm
import sklearn.model_selection as skms
import sklearn.preprocessing as skp
import random
import librosa, IPython
import librosa.display as lplt
seed = 12
np.random.seed(seed)
from sklearn.metrics import confusion_matrix
import plotly.express as px
import plotly.figure_factory as ff


---

## Load and Inspect Data
Load your dataset and take a first look at its structure.

---


df = pd.read_csv('archive (3)/Data/features_3_sec.csv')
df.head()

print("Dataset has",df.shape)

df.columns

audio_path = 'archive (3)/Data/genres_original/blues/blues.00000.wav'
audio_data, sr = librosa.load(audio_path,sr=None)

plt.figure(figsize=(15,5))
lplt.waveshow(audio_data, sr=sr)  # Updated function
plt.title("Waveform")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.show()

# plot zoomed audio wave 
start = 1000
end = 1200
plt.figure(figsize=(16,4))
plt.plot(audio_data[start:end])
plt.show()

---

## Exploratory Data Analysis (EDA)
Visualize and summarize the data to uncover patterns and insights.

---


# Computing the Correlation Matrix
spike_cols = [col for col in df.columns if 'mean' in col]
corr = df[spike_cols].corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))  # Changed from np.bool

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(25, 15))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(0, 25, as_cmap=True, s=90, l=45, n=5)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr,mask=mask,cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True)

plt.title('Correlation Heatmap (for the MEAN variables)', fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.savefig("Corr_Heatmap.png")
plt.show()


The heatmap above shows the pairwise correlations between the mean values of various audio features. Darker shades of red indicate stronger negative correlations, while lighter shades represent weaker or positive correlations. This helps identify relationships between features, such as highly correlated MFCC coefficients.

x = df[["label", "tempo"]]

fig, ax = plt.subplots(figsize=(16, 8));
sns.boxplot(x = "label", y = "tempo", data = x, palette = 'husl');

plt.title('BPM Boxplot for Genres', fontsize = 20)
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 10);
plt.xlabel("Genre", fontsize = 15)
plt.ylabel("BPM", fontsize = 15)
plt.savefig("BPM_Boxplot.png")
plt.show()

The boxplot above compares BPM distributions across music genres. Each box shows the median, quartiles, and spread of BPM values, while dots represent outliers. This visualization highlights differences in tempo characteristics among genres.

# map labels to index
label_index = dict()
index_label = dict()
for i, x in enumerate(df.label.unique()):
    label_index[x] = i
    index_label[i] = x
print(label_index)
print(index_label)

---

# Data Preparation and Model Evaluation

Preparing the audio dataset and evaluating model performance for music genre classification. Data preparation involves cleaning, transforming, and engineering audio features to ensure compatibility with classification models. Model evaluation uses appropriate metrics (e.g., accuracy, precision, recall, F1-score) to assess the model’s ability to correctly identify the genre of a given sound sample.

---

df.label = [label_index[l] for l in df.label]
df.label

# shuffle samples
df_shuffle = df.sample(frac=1, random_state=seed).reset_index(drop=True)

# remove irrelevant columns
df_shuffle.drop(['filename', 'length'], axis=1, inplace=True)
df_y = df_shuffle.pop('label')
df_X = df_shuffle

# split into train dev and test
X_train, df_test_valid_X, y_train, df_test_valid_y = skms.train_test_split(df_X, df_y, train_size=0.7, random_state=seed, stratify=df_y)
X_dev, X_test, y_dev, y_test = skms.train_test_split(df_test_valid_X, df_test_valid_y, train_size=0.66, random_state=seed, stratify=df_test_valid_y)

print(f"Train set has {X_train.shape[0]} records out of {len(df_shuffle)} which is {round(X_train.shape[0]/len(df_shuffle)*100)}%")
print(f"Dev set has {X_dev.shape[0]} records out of {len(df_shuffle)} which is {round(X_dev.shape[0]/len(df_shuffle)*100)}%")
print(f"Test set has {X_test.shape[0]} records out of {len(df_shuffle)} which is {round(X_test.shape[0]/len(df_shuffle)*100)}%")

scaler = skp.StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_dev = pd.DataFrame(scaler.transform(X_dev), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)

---

### Model Training with TensorFlow and Keras

This section defines and trains a neural network for music genre classification using TensorFlow and Keras. The model includes an early stopping callback that halts training once validation accuracy exceeds a predefined threshold, preventing overfitting and saving computation time. Training history is plotted to visualize performance over epochs.

---

import tensorflow as tf
print("TF version:-", tf.__version__)
import keras as k
tf.random.set_seed(seed)

ACCURACY_THRESHOLD = 0.94

class myCallback(k.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy') > ACCURACY_THRESHOLD):
            print("\n\nStopping training as we have reached %2.2f%% accuracy!" %(ACCURACY_THRESHOLD*100))   
            self.model.stop_training = True

def trainModel(model, epochs, optimizer):
    batch_size = 128
    callback = myCallback()
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']  # fixed here
    )
    return model.fit(
        X_train, y_train,
        validation_data=(X_dev, y_dev),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[callback]
    )

def plotHistory(history):
    print("Max. Validation Accuracy",max(history.history["val_accuracy"]))
    pd.DataFrame(history.history).plot(figsize=(12,6))
    plt.show()

model_1 = k.models.Sequential([
    k.layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    k.layers.Dense(128, activation='relu'),
    k.layers.Dense(64, activation='relu'),
    k.layers.Dense(10, activation='softmax'),
])
print(model_1.summary())
model_1_history = trainModel(model=model_1, epochs=11, optimizer='adam')

plotHistory(model_1_history)

model_2 = k.models.Sequential([
    k.layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    k.layers.Dropout(0.2),
    
    k.layers.Dense(256, activation='relu'),
    k.layers.Dropout(0.2),

    k.layers.Dense(128, activation='relu'),
    k.layers.Dropout(0.2),

    k.layers.Dense(64, activation='relu'),
    k.layers.Dropout(0.2),

    k.layers.Dense(10, activation='softmax'),
])
print(model_2.summary())
model_2_history = trainModel(model=model_2, epochs=20, optimizer='adam')

plotHistory(model_2_history)


model_3 = k.models.Sequential([
    k.layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    k.layers.Dropout(0.2),
    
    k.layers.Dense(256, activation='relu'),
    k.layers.Dropout(0.2),

    k.layers.Dense(128, activation='relu'),
    k.layers.Dropout(0.2),

    k.layers.Dense(64, activation='relu'),
    k.layers.Dropout(0.2),

    k.layers.Dense(10, activation='softmax'),
])
print(model_3.summary())
model_3_history = trainModel(model=model_3, epochs=290, optimizer='sgd')

plotHistory(model_3_history)


model_4 = k.models.Sequential([
    k.layers.Dense(1024, activation='relu', input_shape=(X_train.shape[1],)),
    k.layers.Dropout(0.3),
    
    k.layers.Dense(512, activation='relu'),
    k.layers.Dropout(0.3),

    k.layers.Dense(256, activation='relu'),
    k.layers.Dropout(0.3),

    k.layers.Dense(128, activation='relu'),
    k.layers.Dropout(0.3),

    k.layers.Dense(64, activation='relu'),
    k.layers.Dropout(0.3),

    k.layers.Dense(10, activation='softmax'),
])
print(model_4.summary())
model_4_history = trainModel(model=model_4, epochs=30, optimizer='rmsprop')

plotHistory(model_4_history)


test_loss, test_acc  = model_4.evaluate(X_test, y_test, batch_size=128)
print("The test Loss is :",test_loss)
print("\nThe Best test Accuracy is :",test_acc*100)

---

### Spectrogram Generation and CNN Training

This section converts audio files into mel-spectrograms, optionally applying data augmentation through time and frequency masking to improve model generalization. The processed spectrograms are used to train a Convolutional Neural Network (CNN) for music genre classification. The model architecture includes multiple convolutional and pooling layers, dropout for regularization, and early stopping to prevent overfitting. Performance is evaluated on a held-out test set.

---

def create_spectrogram(file_path, img_size=(128, 128), augment=False):
    try:
        y, sr = librosa.load(file_path, duration=30)  # Load first 30 seconds
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)  # Convert to dB
        # Normalize to [0, 1]
        mel_spec_db = (mel_spec_db - mel_spec_db.min()) / (mel_spec_db.max() - mel_spec_db.min())
        mel_spec_db = tf.image.resize(mel_spec_db[..., np.newaxis], img_size).numpy().squeeze()
        
        if augment:
            # Milder time and frequency masking (5% instead of 10%)
            if np.random.rand() > 0.5:
                max_mask = int(mel_spec_db.shape[1] * 0.05)
                t_start = np.random.randint(0, mel_spec_db.shape[1] - max_mask)
                mel_spec_db[:, t_start:t_start + max_mask] = 0
            if np.random.rand() > 0.5:
                max_mask = int(mel_spec_db.shape[0] * 0.05)
                f_start = np.random.randint(0, mel_spec_db.shape[0] - max_mask)
                mel_spec_db[f_start:f_start + max_mask, :] = 0
        
        return mel_spec_db
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# Load audio files from genres_original
data_path = 'archive (3)/Data'
genres_path = os.path.join(data_path, 'genres_original')
data = []
labels = []
img_size = (128, 128)
genres = os.listdir(genres_path)

for genre in genres:
    genre_path = os.path.join(genres_path, genre)
    for file in os.listdir(genre_path):
        if file.endswith('.wav'):
            file_path = os.path.join(genre_path, file)
            spec = create_spectrogram(file_path, img_size, augment=True)
            if spec is not None:
                data.append(spec)
                labels.append(genre)

# Convert to arrays
X_cnn = np.array(data)[..., np.newaxis]  # Add channel dimension
y_cnn = np.array([label_index[l] for l in labels])  # Reuse label_index

# Split data (70% train, ~20% dev, ~10% test)
X_train_cnn, X_test_valid_cnn, y_train_cnn, y_test_valid_cnn = train_test_split(
    X_cnn, y_cnn, train_size=0.7, random_state=42, stratify=y_cnn
)
X_dev_cnn, X_test_cnn, y_dev_cnn, y_test_cnn = train_test_split(
    X_test_valid_cnn, y_test_valid_cnn, train_size=0.66, random_state=42, stratify=y_test_valid_cnn
)

print(f"CNN Train set: {X_train_cnn.shape[0]} samples")
print(f"CNN Dev set: {X_dev_cnn.shape[0]} samples")
print(f"CNN Test set: {X_test_cnn.shape[0]} samples")

from tensorflow.keras import layers, models

# Define improved CNN
cnn_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.4),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(10, activation='softmax')
])

# Compile with lower learning rate
cnn_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
cnn_model.summary()

# Train with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
cnn_history = cnn_model.fit(
    X_train_cnn, y_train_cnn,
    validation_data=(X_dev_cnn, y_dev_cnn),
    epochs=50, batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate on test set
test_loss_cnn, test_acc_cnn = cnn_model.evaluate(X_test_cnn, y_test_cnn)
print(f"Improved CNN Test Accuracy: {test_acc_cnn*100:.2f}%")

# Generate predictions
y_pred_cnn = cnn_model.predict(X_test_cnn).argmax(axis=1)

# Classification report
print("CNN Classification Report:")
print(classification_report(y_test_cnn, y_pred_cnn, target_names=label_index.keys()))

plt.style.use('seaborn-v0_8')  # or just 'seaborn' in older versions
sns.set_palette("husl")

# Predictions for Model_4 (tabular)
y_pred_model_4 = model_4.predict(X_test, batch_size=128).argmax(axis=1)

# Ensure consistent label names (reuse label_index from tabular code)
genres = list(label_index.keys())

# Create subplots for confusion matrices
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

models_data = [
    ("Dense Neural Network (Model_4)", y_test, y_pred_model_4),
    ("Simple CNN", y_test_cnn, y_pred_cnn)
]

# Plot confusion matrices
for idx, (name, y_true, y_pred) in enumerate(models_data):
    cm = confusion_matrix(y_true, y_pred)
    
    # Create heatmap
    sns.heatmap(cm, 
                annot=True,           # Show numbers in cells
                fmt='d',              # Integer format
                cmap='Blues',         # Color scheme
                xticklabels=genres,   # X-axis labels
                yticklabels=genres,   # Y-axis labels
                ax=axes[idx],         # Specify subplot
                cbar_kws={'shrink': 0.8})  # Colorbar size
    
    axes[idx].set_title(f'Confusion Matrix: {name}', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Predicted Genre', fontsize=12)
    axes[idx].set_ylabel('True Genre', fontsize=12)
    axes[idx].tick_params(axis='x', rotation=45)
    axes[idx].tick_params(axis='y', rotation=0)

plt.tight_layout()
plt.show()

# Training history comparison
history_df_model_4 = pd.DataFrame(model_4_history.history)
history_df_cnn = pd.DataFrame(cnn_history.history)

# Create training history plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot accuracy
ax1.plot(history_df_model_4.index, history_df_model_4['accuracy'], 
         label='Model_4 Train', linewidth=2, marker='o', markersize=4)
ax1.plot(history_df_model_4.index, history_df_model_4['val_accuracy'], 
         label='Model_4 Validation', linewidth=2, marker='s', markersize=4)
ax1.plot(history_df_cnn.index, history_df_cnn['accuracy'], 
         label='CNN Train', linewidth=2, marker='^', markersize=4)
ax1.plot(history_df_cnn.index, history_df_cnn['val_accuracy'], 
         label='CNN Validation', linewidth=2, marker='D', markersize=4)

ax1.set_title('Model Accuracy Comparison', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Accuracy', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 1)

# Plot loss (if available)
if 'loss' in history_df_model_4.columns:
    ax2.plot(history_df_model_4.index, history_df_model_4['loss'], 
             label='Model_4 Train Loss', linewidth=2, marker='o', markersize=4)
    ax2.plot(history_df_model_4.index, history_df_model_4['val_loss'], 
             label='Model_4 Val Loss', linewidth=2, marker='s', markersize=4)
    ax2.plot(history_df_cnn.index, history_df_cnn['loss'], 
             label='CNN Train Loss', linewidth=2, marker='^', markersize=4)
    ax2.plot(history_df_cnn.index, history_df_cnn['val_loss'], 
             label='CNN Val Loss', linewidth=2, marker='D', markersize=4)
    
    ax2.set_title('Model Loss Comparison', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('Loss', fontsize=12)
    ax2.legend()
    ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary statistics
print("="*60)
print("MODEL PERFORMANCE SUMMARY")
print("="*60)
print(f"Dense Neural Network (Model_4) Test Accuracy: {test_acc*100:.2f}%")
print(f"Simple CNN Test Accuracy: {test_acc_cnn*100:.2f}%")
print()

# Detailed classification reports
for name, y_true, y_pred in models_data:
    print(f"\nClassification Report - {name}")
    print("-" * 50)
    print(classification_report(y_true, y_pred, target_names=genres, digits=3))

# Performance comparison bar chart
model_names = ['Dense NN\n(Model_4)', 'Simple CNN']
accuracies = [test_acc * 100, test_acc_cnn * 100]

plt.figure(figsize=(10, 6))
bars = plt.bar(model_names, accuracies, 
               color=['skyblue', 'lightcoral'], 
               edgecolor='black', 
               linewidth=1.5,
               alpha=0.8)

# Add value labels on bars
for bar, acc in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             f'{acc:.2f}%', ha='center', va='bottom', fontweight='bold', fontsize=12)

plt.title('Model Accuracy Comparison', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('Test Accuracy (%)', fontsize=12)
plt.ylim(0, max(accuracies) * 1.1)
plt.grid(axis='y', alpha=0.3)

# Add a horizontal line for average performance
avg_acc = np.mean(accuracies)
plt.axhline(y=avg_acc, color='red', linestyle='--', alpha=0.7, 
            label=f'Average: {avg_acc:.2f}%')
plt.legend()

plt.tight_layout()
plt.show()

# Additional insight: Per-class accuracy
print("\n" + "="*60)
print("PER-CLASS PERFORMANCE ANALYSIS")
print("="*60)

for name, y_true, y_pred in models_data:
    print(f"\n{name}:")
    print("-" * len(name))
    
    cm = confusion_matrix(y_true, y_pred)
    per_class_acc = cm.diagonal() / cm.sum(axis=1) * 100
    
    for genre, acc in zip(genres, per_class_acc):
        print(f"{genre:15}: {acc:6.2f}%")

# Model Performance Comparison: Dense Neural Network vs. Simple CNN  

## Key Observations  

1. **Training and Validation Accuracy**  
   - **Dense Neural Network (Model_A)** exhibits higher training accuracy compared to the Simple CNN, suggesting better learning capability on the training data.  
   - However, the validation accuracy of Model_A is slightly lower than its training accuracy, indicating mild overfitting.  
   - **Simple CNN** shows more stable performance with closer alignment between training and validation accuracy, implying better generalization.  

2. **Confusion Matrix Analysis**  
   - **Dense Neural Network (Model_4)** demonstrates strong performance for genres like `reggae`, `pop`, `metal`, and `classical`, with high true positive rates (e.g., 98 for `reggae`, 102 for `metal`).  
   - Misclassifications are observed for `rock` (often confused with `metal` and `reggae`) and `blues` (sometimes misclassified as `jazz` or `metal`).  
   - **Simple CNN** struggles with smaller datasets or overlapping features, as seen in lower true positives (e.g., only 6 correct for `jazz` and 10 for `classical`). It also shows higher misclassification rates across genres like `hiphop` and `disco`.  

3. **Overall Performance**  
   - The Dense Neural Network achieves higher accuracy for most genres, likely due to its capacity to capture complex patterns in the data.  
   - The Simple CNN, while less accurate, generalizes better to unseen data (as seen in the validation curves) and may be more efficient for smaller or noisier datasets.  

## Conclusion  
For tasks requiring high precision (e.g., music genre classification), the **Dense Neural Network** is preferable despite minor overfitting. However, if computational efficiency and generalization are priorities, the **Simple CNN** offers a balanced alternative. Further tuning (e.g., regularization for Model_A or data augmentation for the CNN) could enhance their performance.  


